# Menganalisis risiko gagal bayar peminjam

Tugasmu adalah menyiapkan laporan untuk divisi kredit suatu bank. Kamu akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporanmu akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.


Tujuan dari proyek: mengetahui pengaruh status perkawinan dan jumlah anak dari seorang nasabah terhadap probabilitas gagal bayar dalam pelunasan pinjaman

## Buka file data dan baca informasi umumnya.


In [1]:
import pandas as pd
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [2]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Soal 1. Eksplorasi data

**Deskripsi data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - status perkawinan
- `family_status_id` - pengidentifikasi untuk status perkawinan nasabah
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman


In [3]:
df.shape

(21525, 12)

In [4]:
df.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house


MN:Pengalaman kerja nasabah dalam hari dalam bentuk min, perlu memastikan kondisi ini dalam situasi apa


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


MN: Nilai yang hilang ada di kolom days_employed dan total_income. Karena jumlahnya tidak sesuai dengan jumlah entries

In [6]:
missing_df = df[df['days_employed'].isnull()]
missing_df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


MN: betul, nilai yang hilang ada di setiap baris dari kolom days_employed dan total_income. Perlu mengetahui lebih lanjut apakah jumlah baris yang hilang sama di kedua kolom atau tidak

In [7]:
filtered_data = df[(df['days_employed'].isnull()) & (df['total_income'].isnull())]
filtered_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


MN: check jumlah baris yang hilang di kolom total_income

In [8]:
df.isnull().sum().sort_values(ascending=False) / df.shape[0] * 100

days_employed       10.099884
total_income        10.099884
children             0.000000
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
purpose              0.000000
dtype: float64

MN:
Jumlah baris dalam tabel yang difilter sama dengan jumlah nilai yang hilang. Dari sisi persentase terdapat 10% yang hilang dari data pada kolom 'days_employed' dan 'total_income'. Dari besarannya masih tergolong rendah di bawah 50%. Kemungkinan yang berpengaruh ke 'total_income' ada data pada kolom 'children'.

In [9]:
nasabah_tidak_lengkap = df[df['gender'].isnull() | df['total_income'].isnull()]
nasabah_tidak_lengkap

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [10]:
nasabah_tidak_lengkap.count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

MN: jumlah nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang memiliki pola yang teratur, data di setiap masing-masing kolom jumlahnya sama.

In [11]:
df.count()

children            21525
days_employed       19351
dob_years           21525
education           21525
education_id        21525
family_status       21525
family_status_id    21525
gender              21525
income_type         21525
debt                21525
total_income        19351
purpose             21525
dtype: int64

MN: distribusi antara dataset dengan distribusi tabel yang telah difilter memiliki kemiripan, artinya pola data untuk sementara masih teratur.

## Transformasi data


In [12]:
education_value = df['education'].unique()
education_value

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [13]:
df['education'] = df['education']. str.lower()

In [14]:
education_value = df['education'].unique()
education_value

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [15]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

MN: terdapat jumlah anak dalam nilai negatif (0.2%), dan terdapat jumlah anak sebanyak 20 (0.3%). Jika dilihat dari persentasenya tergolong rendah, tidak sampai 1%. Bisa saja terdapat kesalahan input data, misalnya 20 maksudnya 2 anak, demikian juga 1 anak. Berikutnya dapat mengganti 20 menjadi 2 dan -1 menjadi 1.

In [16]:
df['children'] = df['children'].replace({-1:1, 20:2})

In [17]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [18]:
days_employed_data = df['days_employed'].describe()
days_employed_data

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [19]:
df['days_employed'] = df['days_employed'].abs()

In [20]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [21]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [22]:
df['years_employed'] = df['days_employed']/365*100

In [23]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2311.691240
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1102.685960
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1540.663729
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1130.067728
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,93223.581383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,1240.908675
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,94229.425789
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,578.999147
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,852.734714


In [24]:
df['days_employed'].median()

2194.220566878695

In [25]:
df[df['years_employed']>40]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2311.691240
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1102.685960
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1540.663729
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1130.067728
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,93223.581383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,1240.908675
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,94229.425789
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,578.999147
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,852.734714


In [26]:
df.loc[df['days_employed']>40, 'days_employed']=2190

In [27]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,2190.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2311.691240
1,1,2190.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1102.685960
2,0,2190.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1540.663729
3,3,2190.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1130.067728
4,0,2190.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,93223.581383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,2190.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,1240.908675
21521,0,2190.0,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,94229.425789
21522,1,2190.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,578.999147
21523,3,2190.0,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,852.734714


MN: jumlah data days_employed dalam bentuk negatif cukup besar, nilai negatif diberikan perlakuan menjadi positif dan diganti nilainya dengan nilai median untuk days_employed yang melebihi lama kerja hingga seseorang pensiun.

In [28]:
dob_years_counts = df['dob_years'].value_counts() / df.shape[0]*100
dob_years_counts

35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2.601626
36    2.578397
44    2.541231
29    2.531940
30    2.508711
48    2.499419
37    2.494774
50    2.387921
43    2.383275
32    2.369338
49    2.360046
28    2.336818
45    2.308943
27    2.290360
56    2.262485
52    2.248548
47    2.229965
54    2.225319
46    2.206736
58    2.141696
57    2.137050
53    2.132404
51    2.081301
59    2.062718
55    2.058072
26    1.895470
60    1.751452
25    1.658537
61    1.649245
62    1.635308
63    1.249710
64    1.231127
24    1.226481
23    1.180023
65    0.901278
66    0.850174
22    0.850174
67    0.775842
21    0.515679
0     0.469222
68    0.459930
69    0.394890
70    0.301974
71    0.269454
20    0.236934
72    0.153310
19    0.065041
73    0.037166
74    0.027875
75    0.004646
Name: dob_years, dtype: float64

MN: terdapat dob_years yang nilai 0, dan nilai 0 akan diganti dengan nilai median.

In [29]:
average_age_by_gender = df.groupby('gender')['dob_years'].median()

def replace_age(row):
    if row['dob_years']==0:
        return average_age_by_gender[row['gender']]
    else:
        return row['dob_years']
df['dob_years'] = df.apply(replace_age, axis=1)
df['dob_years'] = df['dob_years'].astype(int)

In [30]:
df['dob_years'].value_counts()

40    638
44    619
35    617
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [31]:
family_status_counts = df['family_status'].value_counts()
family_status_counts

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [32]:
gender_counts = df['gender'].value_counts()
gender_counts

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [33]:
gender_counts = df.drop(df[df['gender']=='XNA'].index, inplace=True)

In [34]:
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [35]:
income_type_counts = df['income_type'].value_counts()
income_type_counts

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [36]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,NaN
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,NaN
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,NaN
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education,NaN
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated,NaN
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation,NaN
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate,NaN


In [37]:
data_fixed = df.drop_duplicates()
data_fixed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,2190.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2311.691240
1,1,2190.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1102.685960
2,0,2190.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1540.663729
3,3,2190.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1130.067728
4,0,2190.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,93223.581383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,2190.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,1240.908675
21521,0,2190.0,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,94229.425789
21522,1,2190.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,578.999147
21523,3,2190.0,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,852.734714


In [38]:
data_fixed[data_fixed.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed


In [39]:
data_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21453 non-null  object 
 12  years_employed    19350 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.3+ MB


MN: jumlah baris sekarang berkurang 1 karena nilai gender XNA sudah dibuang. Kemudian terdapat data duplikat, setelah dirapikan kembali terdapat perubahan jumlah entries dengan persentase perubahannya adalah sebesar 0.3%

# Bekerja dengan nilai yang hilang

MN: menggunakan dictionary berdasarkan kolom, karena ada 2 kolom yang menggunakan ID, yaitu education dan family_status.

In [40]:
education_id_dict = data_fixed.groupby('education_id')['education'].unique().to_dict()
family_status_id_dict = data_fixed.groupby('family_status_id')['family_status'].unique().to_dict()
combine_dict = {'education_id':education_id_dict, 'family_status_id':family_status_id_dict}
combine_dict

{'education_id': {0: array(["bachelor's degree"], dtype=object),
  1: array(['secondary education'], dtype=object),
  2: array(['some college'], dtype=object),
  3: array(['primary education'], dtype=object),
  4: array(['graduate degree'], dtype=object)},
 'family_status_id': {0: array(['married'], dtype=object),
  1: array(['civil partnership'], dtype=object),
  2: array(['widow / widower'], dtype=object),
  3: array(['divorced'], dtype=object),
  4: array(['unmarried'], dtype=object)}}

### Memperbaiki nilai yang hilang di `total_income`

In [41]:
def category_age(age):
    if age < 18:
        return 'under 18'
    if age < 30:
        return '18-29'
    if age < 40:
        return '30-39'
    if age < 50:
        return '40-49'
    if age < 60:
        return '50-59'
    else:
        return '60 and above'

In [42]:
category_age(33)

'30-39'

In [43]:
data_fixed_copy = data_fixed.copy()
data_fixed_copy['age_category'] = data_fixed_copy['dob_years'].apply(category_age)

In [44]:
data_fixed_copy['age_category'].head(10)

0    40-49
1    30-39
2    30-39
3    30-39
4    50-59
5    18-29
6    40-49
7    50-59
8    30-39
9    40-49
Name: age_category, dtype: object

In [45]:
data_fixed = data_fixed_copy.dropna()
data_fixed.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_category
0,1,2190.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2311.691240,40-49
1,1,2190.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1102.685960,30-39
2,0,2190.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1540.663729,30-39
3,3,2190.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1130.067728,30-39
4,0,2190.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,93223.581383,50-59


In [46]:
avg_data_fixed = data_fixed.groupby('income_type')['total_income'].mean()
avg_data_fixed

income_type
business                       32386.741818
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [47]:
median_data_fixed = data_fixed.groupby('income_type')['total_income'].median()
median_data_fixed

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [48]:
avg2_data_fixed = data_fixed.groupby('education')['total_income'].mean()
avg2_data_fixed

education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29040.132990
Name: total_income, dtype: float64

In [49]:
median2_data_fixed = data_fixed.groupby('education')['total_income'].median()
median2_data_fixed

education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25608.7945
Name: total_income, dtype: float64

MN: setelah mempertimbangkan dua faktor yang mempengaruhi pendapatan. Selanjutnya akan menggunakan data median untuk mengisi nilai yang hilang karena nilai kategori entrepreneur memiliki rentang nilai yang cukup jauh dari yang lainnya.

In [50]:
def fill_missing_income(row):
    if pd.isnull(row['total_income']):
        return median_data_fixed[row['income_type']]
    return row['total_income']

In [51]:
data_fixed['total_income'] = data_fixed.apply(fill_missing_income, axis=1)

/tmp/ipykernel_27/1377184912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fixed['total_income'] = data_fixed.apply(fill_missing_income, axis=1)


In [52]:
data_fixed['total_income'].head()

0    40620.102
1    17932.802
2    23341.752
3    42820.568
4    25378.572
Name: total_income, dtype: float64

In [53]:
data_fixed['total_income'].isnull().any()

False

In [54]:
data_fixed['total_income'].shape

(19350,)

In [55]:
data_fixed['income_type'].shape

(19350,)

MN: Jumlah entri di kolom total_income sama dengan jumlah entri di kolom lain seperti income_type.

###  Memperbaiki nilai di `days_employed`

In [56]:
median_data_fixed_income_type = data_fixed.groupby('income_type')['days_employed'].median()
median_data_fixed_income_type

income_type
business                       2190.0
civil servant                  2190.0
employee                       2190.0
entrepreneur                   2190.0
paternity / maternity leave    2190.0
retiree                        2190.0
student                        2190.0
unemployed                     2190.0
Name: days_employed, dtype: float64

In [57]:
median_data_fixed_total_income = data_fixed.groupby('total_income')['days_employed'].median()
median_data_fixed_total_income

total_income
3306.762      2190.0
3392.845      2190.0
3418.824      2190.0
3471.216      2190.0
3503.298      2190.0
               ...  
273809.483    2190.0
274402.943    2190.0
276204.162    2190.0
352136.354    2190.0
362496.645    2190.0
Name: days_employed, Length: 19347, dtype: float64

In [58]:
median_data_fixed_age_category = data_fixed.groupby('age_category')['days_employed'].median()
median_data_fixed_age_category

age_category
18-29           2190.0
30-39           2190.0
40-49           2190.0
50-59           2190.0
60 and above    2190.0
Name: days_employed, dtype: float64

In [59]:
mean_data_fixed_income_type = data_fixed.groupby('income_type')['days_employed'].mean()
mean_data_fixed_income_type

income_type
business                       2189.057014
civil servant                  2188.361246
employee                       2189.137171
entrepreneur                   2190.000000
paternity / maternity leave    2190.000000
retiree                        2190.000000
student                        2190.000000
unemployed                     2190.000000
Name: days_employed, dtype: float64

In [60]:
mean_data_fixed_total_income = data_fixed.groupby('total_income')['days_employed'].mean()
mean_data_fixed_total_income

total_income
3306.762      2190.0
3392.845      2190.0
3418.824      2190.0
3471.216      2190.0
3503.298      2190.0
               ...  
273809.483    2190.0
274402.943    2190.0
276204.162    2190.0
352136.354    2190.0
362496.645    2190.0
Name: days_employed, Length: 19347, dtype: float64

In [61]:
mean_data_fixed_age_category = data_fixed.groupby('age_category')['days_employed'].mean()
mean_data_fixed_age_category

age_category
18-29           2190.000000
30-39           2187.888190
40-49           2189.123597
50-59           2190.000000
60 and above    2190.000000
Name: days_employed, dtype: float64

MN: Selanjutnya akan menggunakan median untuk mengisi nilai yang hilang, karena antara mean dan median belum cukup mendekati/simetris.

In [62]:
def missing_days_employed(row):
    if pd.isnull(row['days_employed']):
        return median_data_fixed_income_type[row['age_category']]
    return row['days_employed']

In [63]:
data_fixed['days_employed'].isnull().sum()

0

In [64]:
data_fixed['days_employed'] = data_fixed.apply(missing_days_employed, axis=1)

/tmp/ipykernel_27/3463110625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fixed['days_employed'] = data_fixed.apply(missing_days_employed, axis=1)


In [65]:
data_fixed['days_employed'].isnull().sum()

0

In [66]:
data_fixed['days_employed'].count()

19350

## Pengkategorian data


In [67]:
educations_counts = data_fixed['education'].value_counts()
income_tye_counts = data_fixed['income_type'].value_counts()

print(educations_counts)
print(income_type_counts)

secondary education    13693
bachelor's degree       4716
some college             674
primary education        261
graduate degree            6
Name: education, dtype: int64
employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


In [68]:
educations_unique = data_fixed['education'].unique()
income_type_unique = data_fixed['income_type'].unique()

print(educations_unique)
print(income_type_unique)

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']
['employee' 'retiree' 'business' 'civil servant' 'unemployed' 'student'
 'entrepreneur' 'paternity / maternity leave']


In [69]:
def categorize_data(education, income_type):
    education_category = ''
    income_type_category = ''
    
    if education in["bachelor's degree", 'graduate degree']:
        education_category = 'Higher Education'
    elif education == 'secondary education':
        education_category = 'Secondary Education'
    elif education == 'some college':
        education_category = 'Some College'
    else:
        education = 'Primary Education'
    
    if income_type in['employee', 'business', 'civil servant']:
        income_type_category = 'Employed'
    elif income_type == 'retiree':
        income_type_category = 'Retired'
    elif income_type == 'unemployed':
        income_type_category = 'Unemployed'
    elif income_type == 'student':
        income_type_category = 'Student'
    else:
        income_type_category = 'Other'
        
    return education_category, income_type_category
    

In [70]:
data_fixed['education_category'], data_fixed['income_type_category'] = zip(*data_fixed.apply(lambda row: categorize_data(row['education'], row['income_type']), axis=1))
data_fixed['education_category'], data_fixed['income_type_category']

/tmp/ipykernel_27/1979899885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fixed['education_category'], data_fixed['income_type_category'] = zip(*data_fixed.apply(lambda row: categorize_data(row['education'], row['income_type']), axis=1))


(0           Higher Education
 1        Secondary Education
 2        Secondary Education
 3        Secondary Education
 4        Secondary Education
                 ...         
 21520    Secondary Education
 21521    Secondary Education
 21522    Secondary Education
 21523    Secondary Education
 21524    Secondary Education
 Name: education_category, Length: 19350, dtype: object,
 0        Employed
 1        Employed
 2        Employed
 3        Employed
 4         Retired
            ...   
 21520    Employed
 21521     Retired
 21522    Employed
 21523    Employed
 21524    Employed
 Name: income_type_category, Length: 19350, dtype: object)

In [71]:
numeric_columns = ['children', 'total_income', 'days_employed']

In [72]:
numeric_summary = data_fixed[numeric_columns].describe()
numeric_summary

,children,total_income,days_employed
count,19350.000000,19350.000000,19350.000000
mean,0.479638,26787.266688,2189.219353
std,0.753906,16475.822926,41.037450
min,0.000000,3306.762000,24.141633
25%,0.000000,16486.515250,2190.000000
50%,0.000000,23201.873500,2190.000000
75%,1.000000,32547.910750,2190.000000
max,5.000000,362496.645000,2190.000000


MN: Rentang yang akan digunakan adalah children dan total_income

In [73]:
def categorize_numeric(row):
    children_category = ''
    income_category = ''
    
    if row ['children'] == 0:
        children_category = 'No Children'
    elif row ['children'] == 1:
        children_category = '1 Child'
    else:
        chidren_category = '2+ Children'
        
    if row ['total_income'] <= 16486.515:
        income_category = 'Low Income'
    elif row ['total_income'] <= 32547.910:
        income_category = 'Medium Income'
    else:
        income_category = 'High Income'
        
    return children_category, income_category   


In [74]:
data_fixed['children_category'] = data_fixed.apply(categorize_numeric, axis=1)
data_fixed['income_category'] = data_fixed.apply(categorize_numeric, axis=1)


/tmp/ipykernel_27/1824825649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fixed['children_category'] = data_fixed.apply(categorize_numeric, axis=1)
/tmp/ipykernel_27/1824825649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fixed['income_category'] = data_fixed.apply(categorize_numeric, axis=1)


In [75]:
children_distribution = data_fixed['children_category'].value_counts()
income_distribution = data_fixed['income_category'].value_counts()

income_distribution

(No Children, Medium Income)    6292
(No Children, Low Income)       3312
(No Children, High Income)      3105
(1 Child, Medium Income)        2292
(1 Child, High Income)          1126
(, Medium Income)               1090
(1 Child, Low Income)            969
(, High Income)                  607
(, Low Income)                   557
Name: income_category, dtype: int64

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [76]:
children_debt = data_fixed.groupby('children')['debt'].value_counts().unstack()
children_debt

debt,0,1
children,,
0,11757.0,952.0
1,3978.0,409.0
2,1733.0,185.0
3,272.0,22.0
4,31.0,3.0
5,8.0,NaN


In [77]:
children_debt['percentage'] = (children_debt[1]/(children_debt[0]+children_debt[1]))*100
children_debt['percentage']

children
0    7.490755
1    9.323000
2    9.645464
3    7.482993
4    8.823529
5         NaN
Name: percentage, dtype: float64

MN: Semakin banyak anak belum tentu peluang untuk gagal bayar juga tinggi.

**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [78]:
family_debt = data_fixed.groupby('family_status')['debt'].value_counts().unstack()
family_debt

debt,0,1
family_status,,
civil partnership,3395,339
divorced,1007,76
married,10297,846
unmarried,2271,254
widow / widower,809,56


In [79]:
family_debt['percentage'] = (family_debt[1]/(family_debt[0]+family_debt[1]))*100
family_debt['percentage']

family_status
civil partnership     9.078736
divorced              7.017544
married               7.592210
unmarried            10.059406
widow / widower       6.473988
Name: percentage, dtype: float64

**Kesimpulan**

MN: secara data yang terbanyak gagal bayar adalah status married dan paling sedikit adalah nasabah berstatus widow/widower, namun secara persentase yang terbanyak gagal bayar adalah unmarried.

**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [80]:
income_type_debt = data_fixed.groupby('income_type')['debt'].value_counts().unstack()
income_type_debt

debt,0,1
income_type,,
business,4230.0,346.0
civil servant,1233.0,79.0
employee,9051.0,963.0
entrepreneur,1.0,NaN
paternity / maternity leave,NaN,1.0
retiree,3262.0,181.0
student,1.0,NaN
unemployed,1.0,1.0


In [81]:
income_type_debt['percentage'] = (income_type_debt[1]/(income_type_debt[0]+income_type_debt[1]))*100
income_type_debt['percentage']

income_type
business                        7.561189
civil servant                   6.021341
employee                        9.616537
entrepreneur                         NaN
paternity / maternity leave          NaN
retiree                         5.257043
student                              NaN
unemployed                     50.000000
Name: percentage, dtype: float64

**Kesimpulan**

MN: dari secara jumlah data dan besaran persentase employee memiliki peluang gagal bayar tertinggi, meskipun secara persentase unemployed memiliki persentase tertinggi, namun belum cukup mewakili secara data karena jumlah orangnya hanya 1 orang saja.

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [82]:
purpose_debt = data_fixed.groupby('purpose')['debt'].value_counts().unstack()
purpose_debt

debt,0,1
purpose,,
building a property,508,53
building a real estate,536,44
buy commercial real estate,554,43
buy real estate,512,39
buy residential real estate,507,39
buying a second-hand car,403,34
buying my own car,409,43
buying property for renting out,543,45
car,417,37


In [83]:
purpose_debt['percentage'] = (purpose_debt[1]/(purpose_debt[0]+purpose_debt[1]))*100
purpose_debt['percentage']

purpose
building a property                          9.447415
building a real estate                       7.586207
buy commercial real estate                   7.202680
buy real estate                              7.078040
buy residential real estate                  7.142857
buying a second-hand car                     7.780320
buying my own car                            9.513274
buying property for renting out              7.653061
car                                          8.149780
car purchase                                 9.307876
cars                                         8.788599
construction of own property                 6.428571
education                                    7.654321
getting an education                         7.888041
getting higher education                    10.769231
going to university                          8.863636
having a wedding                             7.299270
housing                                      7.495741
housing renovation  

**Kesimpulan**

MN: secara persentase tujuan pinjaman to get a supplementary education berpeluang gagal bayar yang tertinggi, namun rentangnya tidak terlalu jauh dari tujua pinjaman yang lainnya. Secara umum rentang gagal bayar berkisar di 5%-11%, kondisi ini tidak terlalu jauh rentangnya. Kemudian tujuan untuk pinjaman yang berkaitan dengan kendaraan dan pendidikan tergolong memiliki persentase gagal bayar yang tinggi dari tujuan yang lain.